In [1]:
from qiskit.algorithms.minimum_eigensolvers import QAOA                             
from qiskit.algorithms.optimizers import COBYLA
from qiskit_aer.primitives import Sampler
from qiskit_finance.applications.optimization import PortfolioOptimization
from qiskit_finance.data_providers import RandomDataProvider
from qiskit_optimization.algorithms import MinimumEigenOptimizer
import datetime

$$
\begin{aligned}
\min_{x \in \{0, 1\}^n}\quad& q x^T \Sigma x - \mu^T x\\
\text{s.t.}\quad& 1^T x = B
\end{aligned}
$$

定义基本的参数，构造数据。

In [2]:
num_assets = 4
data = RandomDataProvider(
    tickers=[f"TICKERS{i}" for i in range(num_assets)],
    start=datetime.datetime(2021, 11, 17),
    end=datetime.datetime(2022, 11, 17),
)
data.run()
mu = data.get_period_return_mean_vector()
sigma = data.get_period_return_covariance_matrix()
q = 0.5
B = num_assets // 2

定义投资组合优化问题。

In [3]:
problem = PortfolioOptimization(mu, sigma, q, B)

将原问题转化为一个二次规划问题。

In [4]:
quadratic_program = problem.to_quadratic_program()
print(quadratic_program)

minimize 0.000498218709983642*x_0^2 - 3.1882442775709215e-05*x_0*x_1 + 4.6815894582219884e-05*x_0*x_2 + 1.6916823931991134e-05*x_0*x_3 + 7.192580454970226e-05*x_1^2 - 1.6249923389122132e-06*x_1*x_2 - 1.2245202406628253e-05*x_1*x_3 + 8.931014235941896e-05*x_2^2 - 4.3173206849120785e-06*x_2*x_3 + 4.7759464933488466e-05*x_3^2 - 0.0015573186401921792*x_0 - 0.0008409042826572975*x_1 - 0.0008119046026683301*x_2 - 0.00018496957710585366*x_3 (4 variables, 1 constraints, 'Portfolio optimization')


利用QAOA求解，得到投资方案。

In [5]:
result = MinimumEigenOptimizer(QAOA(Sampler(), COBYLA())).solve(
    quadratic_program
)
print(result)

fval=-0.0018599608510918417, x_0=1.0, x_1=1.0, x_2=0.0, x_3=0.0, status=SUCCESS


输出全部的可能方案，得到满足约束方案的概率大。

In [6]:
print("投资方案\t\t概率\t目标函数值")
for (
    k,
    v,
) in sorted(
    result.min_eigen_solver_result.eigenstate.binary_probabilities().items(),
    key=lambda x: x[1],
    reverse=True,
):
    x = [int(i) for i in list(reversed(k))]
    print(
        f"{x}\t{v:.4f}\t{problem.to_quadratic_program().objective.evaluate(x):.6f}"
    )

投资方案		概率	目标函数值
[1, 0, 1, 0]	0.1465	-0.001735
[1, 1, 0, 0]	0.1309	-0.001860
[1, 0, 0, 1]	0.1221	-0.001179
[0, 1, 1, 0]	0.1201	-0.001493
[0, 1, 0, 1]	0.0977	-0.000918
[0, 0, 1, 1]	0.0967	-0.000864
[0, 0, 0, 0]	0.0762	0.000000
[1, 1, 1, 1]	0.0674	-0.002674
[0, 1, 0, 0]	0.0225	-0.000769
[1, 0, 0, 0]	0.0225	-0.001059
[0, 0, 0, 1]	0.0215	-0.000137
[1, 0, 1, 1]	0.0215	-0.001859
[1, 1, 1, 0]	0.0166	-0.002537
[0, 1, 1, 1]	0.0137	-0.001647
[1, 1, 0, 1]	0.0137	-0.001992
[0, 0, 1, 0]	0.0107	-0.000723
